# Analyze results for 3D CGAN
Feb 22, 2021

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [ ]:
from ipywidgets import *

In [ ]:
%matplotlib widget

In [ ]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [ ]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [ ]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


In [ ]:
img_size=64
# img_size=128

In [ ]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

### Read validation data

In [9]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=100

for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


## Read data

In [10]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [11]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [12]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202106*')]
n=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(n)

interactive(children=(Dropdown(description='x', options=('20210620_113852_cgan_bs32_nodes8_lr0.0001-vary_fm50_…

In [52]:
result=n.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210620_113852_cgan_bs32_nodes8_lr0.0001-vary_fm50_spec0.01


## Plot Losses

In [53]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [54]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
33921,33921.0,57.0,0.227945,0.225888,0.453833,3.025241,4.157307,1.098274,-3.034031,1.079864,-3.070876,NaN,0.033791,2.259975,-2.486055,-2.948722,0.825756,0.000013,0.000013
33922,33922.0,57.0,0.134001,0.195801,0.329802,3.219066,4.295526,1.060052,-2.752971,1.062216,-3.081280,NaN,0.016407,2.748267,-3.335568,-3.160635,0.823412,0.000013,0.000013
33923,33923.0,57.0,0.202861,0.221179,0.424041,3.075507,4.196239,1.104998,-2.981376,1.081866,-2.907432,NaN,0.015734,2.705071,-2.341373,-3.012329,0.820646,0.000013,0.000013
33924,33924.0,57.0,0.179773,0.243134,0.422906,3.211148,4.337192,1.109466,-2.851337,1.068623,-3.046803,NaN,0.016578,2.492898,-2.514550,-3.146781,0.821733,0.000013,0.000013
33925,33925.0,57.0,0.366987,0.169413,0.536399,2.188873,3.422868,1.054672,-2.931164,1.074867,-3.072756,NaN,0.179323,1.524997,-2.885853,-2.037728,0.826063,0.000013,0.000013
33926,33926.0,57.0,0.153466,0.342004,0.495470,3.846948,4.968709,1.077269,-3.001782,1.072968,-3.028671,NaN,0.044491,3.006906,-1.802003,-3.815507,0.823410,0.000013,0.000013
33927,33927.0,57.0,0.177630,0.237903,0.415533,2.972641,4.120679,1.066059,-2.969698,1.069417,-2.954704,NaN,0.081979,2.304927,-3.746372,-2.891424,0.828723,0.000013,0.000013
33928,33928.0,57.0,0.199351,0.193772,0.393123,2.620822,3.694781,1.035899,-2.226850,1.068346,-3.009676,NaN,0.038060,2.644155,-2.508513,-2.506960,0.826218,0.000013,0.000013
33929,33929.0,57.0,0.105725,0.177163,0.282888,3.207642,4.275024,1.056308,-2.920687,1.041330,-2.066449,NaN,0.011074,3.009235,-2.173890,-3.150555,0.830970,0.000013,0.000013
33930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.053153,-2.836431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [56]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>30)].sort_values(by=['hist_chi']).head(10)

-3.100300025939941


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
20209,20209.0,34.0,0.203007,0.153111,0.356118,3.074500,4.163785,1.071896,-2.887882,1.112927,-3.274905,NaN,0.017389,2.676538,-2.891185,-3.004333,0.832211,0.000025,0.000025
18365,18365.0,31.0,0.206951,0.164804,0.371755,2.594328,3.697608,1.083067,-3.141002,1.067306,-3.255879,NaN,0.020214,1.990640,-2.944499,-2.488064,0.835081,0.000025,0.000025
18454,18454.0,31.0,0.216631,0.213911,0.430542,3.371040,4.485436,1.084773,-2.891356,1.059078,-3.255678,NaN,0.029622,2.384284,-2.268950,-3.318712,0.839933,0.000025,0.000025
18728,18728.0,32.0,0.172398,0.255674,0.428071,2.913367,4.051591,1.096094,-2.939967,1.096988,-3.254844,NaN,0.042129,2.962465,-2.272097,-2.842628,0.825854,0.000025,0.000025
31938,31938.0,54.0,0.202545,0.179691,0.382235,3.084128,4.185385,1.075792,-2.935871,1.076165,-3.254078,NaN,0.025466,2.363185,-3.296060,-3.016571,0.820685,0.000013,0.000013
19604,19604.0,33.0,0.151813,0.306973,0.458787,4.150357,5.259009,1.095345,-3.025568,1.064652,-3.253830,NaN,0.013306,3.064394,-1.858050,-4.129333,0.832278,0.000025,0.000025
18453,18453.0,31.0,0.240442,0.184751,0.425194,3.202212,4.335739,1.114935,-2.881084,1.060536,-3.251451,NaN,0.018592,2.279070,-2.509068,-3.146842,0.841724,0.000025,0.000025
24069,24069.0,41.0,0.194226,0.253001,0.447227,3.812037,4.929135,1.063639,-2.653912,1.081859,-3.249398,NaN,0.053460,3.369115,-1.950829,-3.773690,0.828945,0.000025,0.000025
20988,20988.0,35.0,0.280475,0.109435,0.389910,3.047004,4.276380,1.114330,-3.037111,1.095960,-3.249397,NaN,0.115046,1.941792,-3.206362,-2.953848,0.839612,0.000025,0.000025
20208,20208.0,34.0,0.130035,0.156972,0.287007,3.338141,4.458290,1.086542,-2.867719,1.093795,-3.247861,NaN,0.033607,3.200025,-2.258463,-3.282230,0.882083,0.000025,0.000025


In [57]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [58]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [59]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [60]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1b']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


In [61]:
df

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
0,0,10,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.476507,0.025009,0.000000,0.501516,0.009396,0.976425,...,55.579206,16,0,0,"[0.6772064213821565, 0.11337175353741562, 0.03...","[0.0008751756863119238, 0.003489961268110466, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[5383045.68021488, 474975.03205804736, 149823....","[374333.58645615674, 39509.94180988222, 14809....",0-10
1,0,20,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.322816,0.025009,0.000000,0.347825,0.003545,0.070459,...,55.869008,16,0,0,"[0.6983994965773479, 0.008871030795350253, 0.0...","[0.00010927500349668811, 0.0005766530756777317...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[3770200.873982636, 423678.02140395023, 200653...","[353011.0612188846, 31731.131430247304, 12237....",0-20
2,0,30,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.342703,0.025009,0.000000,0.367712,0.004060,0.001089,...,56.560235,16,0,0,"[0.6999811415903796, 0.00011115074142497777, 4...","[2.6542678494349238e-06, 1.573558555957735e-05...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[334264.2452396784, 124522.44001341215, 107709...","[24359.640221607646, 5865.523125333447, 2143.6...",0-30
3,0,40,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.325908,0.025009,0.000000,0.350918,0.003759,0.111436,...,56.597253,16,0,0,"[0.6994177108038443, 0.002774763103501022, 0.0...","[3.348895920125678e-05, 0.00015278649004349853...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[118543.64903107741, 38825.716899338375, 44136...","[28974.54469142841, 2071.6725740556617, 1695.3...",0-40
4,0,50,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.142408,0.159120,0.000074,0.301602,0.001591,2.586853,...,56.473941,16,5,0,"[0.6952750926215641, 0.019887971850824172, 0.0...","[8.706281001524814e-05, 0.000275046343557171, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[776879.6452108764, 119909.55678720848, 52082....","[93881.24057726914, 11450.408671980616, 4036.2...",0-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406,58,34070,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.035377,2.794408,0.000540,2.830324,0.001536,12.400396,...,45.569294,16,40,0,"[0.691915346817419, 0.02858584651915462, 0.019...","[0.0005653752385894489, 0.0017699530819377351,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[11785650.883408217, 5077772.3939905595, 26911...","[3781831.47706526, 1143579.2018872576, 590332....",58-34070
3407,58,34080,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.068623,1.926348,0.000584,1.995556,0.001762,13.985861,...,46.715982,16,46,0,"[0.6917077183865035, 0.02854283154874879, 0.02...","[0.0006099599765865989, 0.00187365734291509, 0...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[12339324.880494783, 5225958.741073604, 274447...","[4080285.660435182, 1269890.391534296, 640961....",58-34080
3408,58,34090,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.229858,3.831113,0.000885,4.061857,0.003392,18.108377,...,46.796523,16,52,1,"[0.6901370722656173, 0.03347757551666858, 0.02...","[0.0007084906721898073, 0.0021422722717475916,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[13784116.084679345, 5799077.189568704, 306201...","[4605854.593869968, 1396164.3336915611, 716991...",58-34090
3409,58,34100,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.121767,2.695898,0.000486,2.818150,0.001883,13.768695,...,46.730312,16,45,0,"[0.6912379060392175, 0.03034731017339573, 0.02...","[0.0006382100270340258, 0.001963659884415388, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[12620263.033095863, 5326202.26166861, 2816742...","[4252562.97082444, 1280770.383606728, 653451.6...",58-34100


### Slice best steps

In [62]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1b']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [63]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [64]:
df_sliced=w.result
[int(i.split('-')[1]) for i in df_sliced.label.values]

# df_sliced

[]

In [65]:
best_step=[]
df_test=df_merged.copy()
df_test=df_merged[(df_merged.epoch<5000)&(df_merged.epoch>40)]
cut_off=0.9

best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=8,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=8,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[24700 25150 25170 25180 25260 25450 25600 25710 30490 31360 31790 32530
 32540 32590 32600 33580]


array([24700, 25150, 25170, 25180, 25260, 25450, 25600, 25710, 30490,
       31360, 31790, 32530, 32540, 32590, 32600, 33580])

In [66]:
# best_step=[6176]
# best_step= [23985,24570,25155,25740,26325,26910,27495]
# best_step=[int(i.split('-')[1]) for i in df_sliced.label.values]
# best_step=np.arange(40130,40135).astype(int)

In [67]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(16, 61)
[(42, 24700), (42, 25150), (43, 25170), (43, 25180), (43, 25260), (43, 25450), (43, 25600), (43, 25710), (52, 30490), (53, 31360), (54, 31790), (55, 32530), (55, 32540), (55, 32590), (55, 32600), (57, 33580)]


In [68]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1b
2469,42-24700,0.905060,2.951227,0.810776
2514,42-25150,0.851176,1.728845,0.743302
2516,43-25170,0.503533,2.667336,0.410920
2517,43-25180,0.879608,3.438562,0.793943
2525,43-25260,0.872286,4.123661,0.779238
2544,43-25450,0.830346,1.907296,0.735149
2559,43-25600,0.920741,1.729976,0.816407
2570,43-25710,0.383981,2.870406,0.308535
3048,52-30490,2.446783,0.602660,2.344250
3135,53-31360,2.423559,0.573603,2.225903


### Interactive plot

In [69]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type,img_size):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:,0]
                print(images.shape)
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [70]:
interact_manual(f_plot_hist_spec,df=fixed(df_best),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=df_best.step.values),
                img_size=fixed(img_size),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type, img_size)>

In [71]:
best_step

array([24700, 25150, 25170, 25180, 25260, 25450, 25600, 25710, 30490,
       31360, 31790, 32530, 32540, 32590, 32600, 33580])

In [ ]:
sigma=1.1
fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma)
print(fname)
a1=np.load(fname,mmap_mode='r')[-100:]
a1.shape

images=a1[:,0,:,:,0]
f_plot_grid(images[8:16],cols=4,fig_size=(8,4))


### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [ ]:
# fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
fldr=result_dir
print(fldr)
flist=glob.glob(fldr+'/models/checkpoint_*.tar')
len(flist)

In [ ]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [ ]:
best_step

In [ ]:
# ! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.5_H70.0.npy'
np.load(fname,mmap_mode='r').shape